In [1]:
!pip install gradio

In [7]:
import gradio as gr
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, TextDataset, create_optimizer


RuntimeError: Failed to import transformers.models.gpt2.modeling_tf_gpt2 because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


In [6]:
# Load and preprocess dataset
def load_dataset(file_path):
    return TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=128)

dataset = load_dataset("custom_dataset.txt")
# Convert to tf.data.Dataset
def convert_to_tf_dataset(dataset, tokenizer, batch_size):
    def gen():
        for item in dataset:
            inputs = tokenizer(item, return_tensors='tf', padding=True, truncation=True).input_ids
            yield {"input_ids": inputs, "labels": inputs}
    return tf.data.Dataset.from_generator(gen, output_signature={
        "input_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
        "labels": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    }).batch(batch_size)

batch_size = 4
tf_dataset = convert_to_tf_dataset(dataset, tokenizer, batch_size=batch_size)

NameError: name 'TextDataset' is not defined

In [5]:
# Determine the number of steps per epoch
steps_per_epoch = tf.data.experimental.cardinality(tf_dataset).numpy()

# Training parameters
epochs = 3

# Define optimizer
optimizer, _ = create_optimizer(init_lr=5e-5, num_train_steps=steps_per_epoch * epochs, num_warmup_steps=0)

# Custom training loop
for epoch in range(epochs):
    epoch_loss = 0.0  # Initialize as a float
    for batch in tf_dataset:
        with tf.GradientTape() as tape:
            outputs = model(batch["input_ids"], labels=batch["labels"], return_dict=True)
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(batch["labels"], outputs.logits, from_logits=True))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += loss.numpy()  # Convert tensor to float

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {epoch_loss}")

NameError: name 'tf_dataset' is not defined